# Intelligent Systems - Individual Project Assessment
I aim to split the code into 3 sections:
1. Generating and analysing the datset
2. Developing my classification model
3. Training my model

In [604]:
import matplotlib.pyplot as plt
import torch
from torch import nn
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import scipy.io as sio
import numpy as np

In [611]:
# set the selected device for the tensors
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print(torch.cuda.get_device_name(device))
torch.set_default_device(device)

Using device: cuda
NVIDIA GeForce RTX 3050 Laptop GPU


## Dataset

### Data Augmentation
After inspection of the dataset, we have PIL images. Therefore, we will convert these to Tensors.

The values used for the normalisation of data were calculated from the ImageNet training datase

In [606]:
# We perform random transformations to better generalise the training dataset
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(size=(224, 224), antialias=True),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                 std=[0.229, 0.224, 0.225]) 
])
valid_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                 std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                 std=[0.229, 0.224, 0.225])
])

### Downloading and splitting the dataset

In [607]:
# I will download the data from PyTorch's website and use the appropriate data loader
train_dataset = datasets.Flowers102(
    root='flowers102',
    split="train",
    download=True,
    transform=train_transform
)

valid_dataset = datasets.Flowers102(
    root='flowers102',
    split="val",
    download=True,
    transform=valid_transform
)

test_dataset = datasets.Flowers102(
    root='flowers102',
    split="test",
    download=True,
    transform=test_transform
)

# Get the targets and ids
image_labels = sio.loadmat("flowers102/flowers-102/imagelabels")
setids = sio.loadmat("flowers102/flowers-102/setid")

In [608]:
# look at the first training sample
image, label = train_dataset[0]
print(f"Image shape: {image.shape} -> [batch, height, width]")
print(f"Datatype: {image.type}")
print(f"Label: {image_labels['labels'][label]}")
print(f"Device tensor is stored on: {image.device}")

Image shape: torch.Size([3, 224, 224]) -> [batch, height, width]
Datatype: <built-in method type of Tensor object at 0x00000262D4188320>
Label: [77 77 77 ... 62 62 62]
Device tensor is stored on: cpu


## Model
#### Hyperparameters

### Residual Block
As I wish to implement a model based upon a Residual Network, I must first build a *Residual Block*. A basic block consists of two sequential 3x3 convolutional layers and a residual connection, where the input and output dimensions of the layers are the same.

In [618]:
class ResidualBlock(nn.Module):
    
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        
        self.expansion = 4
        
        self.layer1 = nn.Sequential(
            # Typically use odd stride values, giving the benefit of preserving the dimensionality
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU())
        
        # Note that for a residual block, stride=1 in order to maintain spatial dimensionality
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_channels * self.expansion),
            nn.ReLU())
        
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
        
    def forward(self, x):
        residual = x
        out = self.layer1(x)
        out = self.layer2(x)
        
        if self.downsample is not None:
            residual = self.downsample(x)
        
        out += residual
        out = self.relu(out)
        return out

### ResNet Architecture

In [621]:
class MyNN(nn.Module):
    
    def __init__(self, ResidualBlock, layers, image_channels, num_classes):
        """

        :param ResidualBlock: A residual block class
        :param layers: A list stating how many times we wish to use the residual block
        :param image_channels: 
        :param num_classes: 
        """
        super(MyNN, self).__init__()
        
        self.in_channels = 64
        
        self.layer0 = nn.Sequential(
            nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=2)
        )
        
        # Architecture's layers
        self.layer1 = self.make_layer(ResidualBlock, layers[0], out_channels=64, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, layers[1], out_channels=128, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, layers[2], out_channels=128, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, layers[3], out_channels=512, stride=2)
        
        self.avgPool = nn.AdaptiveAvgPool2d((1, 1))
        
        self.fc = nn.Linear(512*4, num_classes)
    
    def forward(self, x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        # Make it the correct dimension
        x = nn.AvgPool2d(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        return x
            
    def make_layer(self, ResidualBlock, n_res_blocks, out_channels, stride):
        downsample = None
        layers = []
        
        if stride != 1 or self.in_channels != out_channels * 4:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels=out_channels*4, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels * 4)
            )
        
        layers.append(ResidualBlock(self.in_channels, out_channels, downsample, stride))
        self.in_channels = out_channels * 4     # 256 in -> 64 out, then map to 64*4 (256) again
        
        for i in range(n_res_blocks - 1):
            layers.append(ResidualBlock(self.in_channels, out_channels))
        
        # Return a sequence of modules that is the same as the layers list (including order)  
        return nn.Sequential(*layers)

In [622]:
def NN50(img_channels=3, num_classes=1000):
    return MyNN(ResidualBlock, [3, 4, 6, 3], img_channels, num_classes)

In [623]:
net = NN50()
print(net)

MyNN(
  (layer0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=1, padding=2, dilation=1, ceil_mode=False)
  )
  (layer1): Sequential(
    (0): ResidualBlock(
      (layer1): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1)), BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (layer2): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (relu): ReLU()
    )
    (1): ResidualBlock(
      (layer1): Sequ

In [624]:
# Get the learnable parameters
params = list(net.parameters())
print(len(params))
print(params[0].size())

134
torch.Size([64, 3, 7, 7])


## Train Network

In [625]:
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [628]:
optimiser = torch.optim.Adam(net.parameters(), lr=1e-3)

first
# try to predict the first training sample

AttributeError: 'int' object has no attribute 'split'